# HW2. Anton Bilchuk

In [57]:
import pandas as pd
import numpy as np

In [58]:
df = pd.read_csv('./data/DR_Demo_Fire_Ins_Loss_only.csv')

In [59]:
pd.set_option('display.max_columns', None)
df.head()

,loss,Exposure,Rating_Class,Sub_Rating_Class,Renewal_class,Sub_Renewal_Class,Property_size,Residents,Commercial,Norm_fire_risk,Norm_monthly_rent,Loan_mortgage,No_claim_Years,Previous_claims,Norm_area_m,Premium_remain,Premium_renew,Renewal_Type,crime_property_type,crime_residents,crime_area,crime_arson,crime_burglary,crime_neighbour_watch,crime_community,crime_risk,Geographical_risk,Weather_risk,ISO,ISO_cat,ISO_desc
0,0.07,3.0,H1,NaN,NaN,NaN,3.0,4.0,NaN,4.314082,758.287544,2.673639,2.251423,0.0,10.0,NaN,0.000000,B,1.209807,1.064398,1.066522,0.939346,1.708534,0.808015,1.040819,1.171366,0.284447,0.313157,5,ISO 5 - Modified or Semi Fire Resistive (MFR o...,which and floors. greater roof _—– modified an...
1,0.13,NaN,M1,NaN,NaN,NaN,6.0,1.0,NaN,4.171214,1060.660172,2.709158,2.084180,NaN,NaN,NaN,86.666667,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.006305,0.889677,6,ISO 6 - Fire Resistive (FR),bearing are walls deck/cover precast and/or co...
2,0.02,NaN,M1,NaN,NaN,NaN,4.0,1.0,NaN,4.171214,1060.660172,2.709158,2.084180,NaN,8.0,NaN,86.666667,B,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.019372,0.276507,1,ISO 1 Frame (combustible walls or roof),*BUR 1B and covers the *Single-ply wood/hardip...
3,0.13,NaN,M1,NaN,NaN,NaN,2.0,1.0,NaN,4.171214,1060.660172,2.709158,2.084180,NaN,12.0,NaN,86.666667,B,2.304699,1.912726,1.463283,1.632539,1.205554,1.999838,1.665336,1.779279,0.085413,0.124679,1,ISO 1 Frame (combustible walls or roof),"frame anchorage are wood. *Less Habitational, ..."
4,0.04,3.0,H1,NaN,NaN,NaN,4.0,3.0,NaN,4.314082,716.709146,2.711775,2.408759,0.0,NaN,100.0,0.000000,B,0.669145,0.430252,0.345801,0.638615,0.567431,0.751454,0.585363,0.729587,5.570830,0.703054,6,ISO 6 - Fire Resistive (FR),is roof deck structural on roof - poured occup...


In [60]:
from sklearn.model_selection import train_test_split

y = df.pop('loss');
X = df
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=0, shuffle=True)
y_train = y_train.values
y_test = y_test.values

In [61]:
numeric_features = [
    'Norm_fire_risk','Norm_monthly_rent','Loan_mortgage','No_claim_Years','Premium_renew',\
    'crime_property_type','crime_residents','crime_area','crime_arson','crime_burglary','crime_neighbour_watch','crime_community',\
    'crime_risk','Geographical_risk','Weather_risk','ISO','Property_size','Residents','Previous_claims','Norm_area_m','Premium_remain','Exposure'
]
text_features = ['ISO_desc']
categorical_features = ['ISO_cat','Rating_Class','Renewal_class','Sub_Renewal_Class','Commercial','Renewal_Type','Sub_Rating_Class']

In [62]:
from sklearn.base import BaseEstimator, TransformerMixin, clone

class StackingTransformer(BaseEstimator, TransformerMixin):
    """A transformer that wraps a RegressorMixin `est` predict function.
    
    It overloads `fit_transform` to do a stacked transform to avoid leakage.
    """
    def __init__(self, est):
        self.est = est
        
    def fit(self, X, y=None):
        if y is None:
            raise ValueError('{}.fit requires y to be not None'.format(self))
        self.est.fit(X, y)
        return self
    
    def transform(self, X):
        return self.est.predict(X)[:, np.newaxis]
    
    def fit_transform(self, X, y=None):
        if y is None:
            raise ValueError('{}.fit requires y to be not None'.format(self))
        self.models_ = []
        out = np.empty_like(y)[:, np.newaxis]
        for train, test in KFold(5, shuffle=True, random_state=0).split(X):
            self.models_.append(clone(self.est).fit(X[train], y[train]))
            out[test, 0] = self.models_[-1].predict(X[test])
        self.est = self.models_[0]
        return out

In [66]:
from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OrdinalEncoder
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import ElasticNet
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import KFold

# 
# Pipelines for data preprocessing
#
numeric_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value=-99999)),
    ])

categorical_pipeline = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore')),
    ('fx_selection', SelectFromModel(ElasticNet(alpha=0.1), max_features=60, threshold=None))
])
text_pipeline = Pipeline(steps=[
    ('bow', CountVectorizer()),
    ('bow_reg', StackingTransformer(ElasticNet(alpha=0.1)))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_pipeline, numeric_features),
        ('cat', categorical_pipeline, categorical_features),
        ('text', text_pipeline, text_features[0])
    ])

# Create model with pipeline and RandomForest prediction model
est = Pipeline(steps=[('preprocessor', preprocessor),
                      ('rf', RandomForestRegressor(criterion='mae', n_jobs=-1))])

est.fit(X_train, y_train)

print("model score: %.3f" % mean_absolute_error(y_test, est.predict(X_test)))

/Users/admin/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


model score: 0.038


## Let's add hyperparameter optimization using BayesSearchCV tool

In [ ]:
from skopt.space import Real, Integer, Categorical
from skopt import BayesSearchCV

param_grid = {
    "rf__n_estimators":      (1, 100),
    "rf__max_depth":         (1,30),
    "rf__min_samples_split": (2, 10),
    "rf__min_samples_leaf":  (1, 10)
}

gs = BayesSearchCV(est, param_grid, cv=3, scoring='neg_mean_absolute_error', 
                   n_iter=24, n_jobs=-1, verbose=10)

gs.fit(X_train, y_train)

In [68]:
print("model with optimized parameters score: %.3f" % mean_absolute_error(y_test, gs.predict(X_test)))

model with optimized parameters score: 0.037
